In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/00 - PROJ (TCC)/PREDIÇÃO ATIVIDADES/event2mind'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

IMPORTS NECESSÁRIOS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from nltk.tokenize import word_tokenize
import nltk.data
nltk.download('punkt')

import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Carrega DataSets

In [ ]:
datasetCompleto = pd.read_csv('event2MindClean.csv')
clusterEmocoes = pd.read_csv('clusterEmotions.csv', sep = ',')

In [ ]:
def removeCaracteres(nomeColuna, dataSet):
  dataSet[nomeColuna]= dataSet[nomeColuna].astype(str)
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` ve", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` s", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("'", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("&", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace(",", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("  ", " ")
  
removeCaracteres('Xemotion_2', datasetCompleto)

In [ ]:
clusterEmocoes.head(5)

,cluster_emotions,Xemotion
0,5,pleased
1,0,smug
2,1,excited
3,1,powerful
4,3,satisfied helping mankind


In [45]:
clusterEmocoes['cluster_emotions'].value_counts()

1    3066
7    2089
3    1291
0    1191
2    1136
4    1117
6     942
5     569
Name: cluster_emotions, dtype: int64

JUNTAR TODAS AS INTENÇOES NUMA ÚNICA LISTA


In [ ]:
#cluster_intents da base intencoesAll

clusterEmocoes['Xemotion_2'] = clusterEmocoes['Xemotion']

dtSetWithCluster = clusterEmocoes[["cluster_emotions", "Xemotion_2"]].join(datasetCompleto.set_index('Xemotion_2'), on = 'Xemotion_2', how = 'left')  


In [ ]:
dtSetWithCluster.head(5)

,cluster_emotions,Xemotion_2,Indice,Xintent,Xemotion,Event,Xsent,Osent,Event_2,Xintent_2
0,5,pleased,773,to have an advantage,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'advantage '
0,5,pleased,773,to fulfill a desire,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...","'fulfill ' , 'desire '"
0,5,pleased,773,to get out of trouble,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...","'get ' , 'trouble '"
0,5,pleased,979,to form something from the pieces,pleased,PersonX connects the ___ together,5.0,5.0,"'personx, ', ',', 'connect, ', ',', ', ', ',',...","'form ' , 'something ' , 'pieces '"
0,5,pleased,979,to complete something,pleased,PersonX connects the ___ together,5.0,5.0,"'personx, ', ',', 'connect, ', ',', ', ', ',',...","'complete ' , 'something '"


In [9]:
#função para criar Polaridade
def polaridade_positiva(x):
  if x >= 4:
    return 1
  else:
    return 0

def polaridade_negativa(x):
  if x <= 2:
    return 1
  else:
    return 0

dtSetWithCluster['pos_polarity_Xsent'] = dtSetWithCluster['Xsent'].apply(polaridade_positiva) #com apply e função de classificação
dtSetWithCluster['neg_polarity_Xsent'] = dtSetWithCluster['Xsent'].apply(polaridade_negativa) #com apply e função de classificação

basePosit = dtSetWithCluster.groupby("cluster_emotions")["pos_polarity_Xsent"].mean()
baseNeg = dtSetWithCluster.groupby("cluster_emotions")["neg_polarity_Xsent"].mean()


In [10]:
basePosit.head(10)

cluster_emotions
0    0.528653
1    0.613730
2    0.562110
3    0.735119
4    0.528986
5    0.559250
6    0.602328
7    0.527020
Name: pos_polarity_Xsent, dtype: float64

In [11]:
baseNeg.head(10)

cluster_emotions
0    0.125460
1    0.097487
2    0.138849
3    0.049211
4    0.107890
5    0.079504
6    0.086809
7    0.125651
Name: neg_polarity_Xsent, dtype: float64

In [12]:
dtSetWithCluster['pos_polarity_Osent'] = dtSetWithCluster['Osent'].apply(polaridade_positiva) #com apply e função de classificação
dtSetWithCluster['neg_polarity_Osent'] = dtSetWithCluster['Osent'].apply(polaridade_negativa) #com apply e função de classificação

basePosit = dtSetWithCluster.groupby("cluster_emotions")["pos_polarity_Osent"].mean()
baseNeg = dtSetWithCluster.groupby("cluster_emotions")["neg_polarity_Osent"].mean()



In [13]:
basePosit.head(10)

cluster_emotions
0    0.552395
1    0.640185
2    0.587770
3    0.766891
4    0.557568
5    0.633313
6    0.644035
7    0.540704
Name: pos_polarity_Osent, dtype: float64

In [14]:
baseNeg.head(10)

cluster_emotions
0    0.261154
1    0.201534
2    0.257794
3    0.096855
4    0.235910
5    0.187424
6    0.183964
7    0.272452
Name: neg_polarity_Osent, dtype: float64

In [15]:
len(dtSetWithCluster['cluster_emotions'].unique())

8

In [16]:
dtSetWithCluster['cluster_emotions'].unique()

array([5, 0, 1, 3, 6, 4, 7, 2])

In [17]:
dtSetWithClusterDedup = dtSetWithCluster[['Xemotion_2','cluster_emotions']]

dtSetWithClusterDedup.drop_duplicates(keep=False, inplace=True)

dtSetWithClusterDedup.head()

,Xemotion_2,cluster_emotions
4,satisfied helping mankind,3
8,vengeance,0
9,good calling attention something important,0
16,wins exam,6
17,better prepared,3


FUNÇÃO PARA CRIAR CONTAGEM DE PALAVRAS POR CLUSTER

In [39]:
def count_words_cluster(df, coluna_cluster):
  
  contagemClusters = pd.DataFrame(columns=['index', 'count', 'cluster'])
  
  num_cluster = len(df[coluna_cluster].unique())

  for i in range(0, num_cluster):
    
    filtro_cluster = df[df[coluna_cluster] == i] 
    #  ******* alterar a coluna ******
    newdf = filtro_cluster.Xemotion_2.str.split(expand=True).stack().value_counts().reset_index(name="count").query("count > 1")
    newdf['cluster'] = i 
    contagemClusters = contagemClusters.append(newdf)

  return contagemClusters

CHAMADA DA FUNÇÃO

In [40]:
dtFrameContagens = count_words_cluster(dtSetWithClusterDedup, 'cluster_emotions')

In [41]:
dtFrameContagens.tail(5)

,index,count,cluster
488,fire,2,7
489,ate,2,7
490,another,2,7
491,country,2,7
492,support,2,7


In [42]:
len(dtFrameContagens['cluster'].unique())

8

In [43]:
dtFrameContagens['cluster'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=object)

In [44]:
export_path = workdir_path + '/contagemClusterEmocoes.csv'

dtFrameContagens.to_csv (export_path, index = False, header=True)

In [48]:
clusterIntentions = pd.read_csv('clusterIntencoes.csv', sep = ',')
clusterIntentions['cluster_intents'].value_counts()

5    6154
1    3649
2    3435
3    3365
7    2870
0    2293
6    2071
4    1954
Name: cluster_intents, dtype: int64